# Appendix: Writing data to Excel sheets

Writing Excel sheets requires the **xlsxwriter module** that comes with the Anaconda package:

In [1]:
import xlsxwriter

In [42]:
# assume that you have gathered the following data,
# e.g. by batch evaluating data files on your hdd:

col_names = [
  'Material name',
  'Tensile strength [MPa]',
  'Youngs modulus [MPa]',
  'Elongation at break [%]',    
]

data = [
  [ 'PEEK 450FC30',  123.2, 12250.1, 1.75 ],
  [ 'Ensinger PVX',  100.7, 10174.8, 2.09 ],
  [ 'Ultraform AX',   89.1,  8546.2, 4.56 ],
  [ 'Evonik F4000',  133.0, 13629.6, 1.98 ],
  [ 'Luvocom XCF30', 176.3, 18357.8, 2.13 ],
]

# writing them to an Excel sheet is as easy as follows

# create a workbook...
workbook  = xlsxwriter.Workbook('./img/lecture10/tensile_test_results_01.xlsx')

# ...  and add a worksheet
# the first argument is the new worksheet's name:
worksheet = workbook.add_worksheet('Data')

# rows and columns are denominated by zero indexed integers.
# e.g. A1 = 0,0 and G7 = 6,6

# write the column names into the first row (index 0)
col = 0
for name in col_names:
    worksheet.write(0, col, name)
    col += 1

# iterate over the data and write it out row by row.
row = 1
for line in data:
    for col in range(len(line)):
        worksheet.write(row, col, line[col])
    row += 1

### add bar graphs 

# create bar new chart of type 'bar'
strength_chart = workbook.add_chart({'type': 'column'})

# add chart title and axis labels
strength_chart.set_title({ 'name': 'Measured tensile strengths of selected materials'})
strength_chart.set_x_axis({'name': 'Material name'})
strength_chart.set_y_axis({'name': 'Tensile strength [MPa]'})

# add data to chart
strength_chart.add_series({
    'values':     '=Data!$B$2:$B$'+str(len(data)+1),
    'categories': '=Data!$A$2:$A$'+str(len(data)+1),
})

# unset legend
strength_chart.set_legend({'none': True})

# create chartsheet for storing the chart
chartsheet_1 = workbook.add_chartsheet('strengths plot')

# insert chart into chartsheet
chartsheet_1.set_chart(strength_chart)

# add bar graphs for Young's modulus and elongation at break
# ...

# close workbook
workbook.close()

Here is how our table and our barchart look like:

![](./img/lecture10/xlsx_table_01.png)  
![](./img/lecture10/xlsx_chart_01.png)

Let's modify the data container structure to also contain errors  
and modify our xlsx writing program to mind the new data structure:

In [43]:
col_names = [
  'Material name',
  'Tensile strength [MPa]',
  'Youngs modulus [MPa]',
  'Elongation at break [%]',    
]

data = [
  [ 'PEEK 450FC30', 
     [ 123.2,     6.7 ], # tensile strength, tensile strength error
     [ 12250.1, 280.1 ], # Young's modulus, Young's modulus error
     [ 1.75,     0.06 ], # elongation at break, ... error
  ],
  [ 'Ensinger PVX',
     [ 100.7,     3.1 ],
     [ 10174.8, 357.0 ],
     [ 2.09,     0.14 ],
  ],
  [ 'Ultraform AX', 
     [ 89.1,     6.8 ],
     [ 8546.2, 287.1 ],
     [ 4.56,     0.7 ],
  ],
  [ 'Evonik F4000',
     [ 133.0,     6.2 ],
     [ 13629.6, 241.8 ],
     [ 1.98,     0.09 ],
  ],
  [ 'Luvocom XCF30',
     [ 176.3,    10.7 ],
     [ 18357.8, 987.6 ],
     [ 1.98,     0.57 ],
  ],
]

# create a workbook...
workbook  = xlsxwriter.Workbook('./img/lecture10/tensile_test_results_02.xlsx')

# ...  and add a worksheet
# the first argument is the new worksheet's name:
worksheet = workbook.add_worksheet('Data')

# write the column names into the first row (index 0)
worksheet.write(0, 0, col_names.pop(0))
col = 1
for name in col_names:
    #worksheet.write(0, col, name)
    worksheet.merge_range(0, col, 0, col+2, name)
    col += 3

# reset row index
# and iterate over the data and write col 1
row = 1
for line in data:
    worksheet.write(row, 0, line.pop(0))
    row += 1

row = 1
for line in data:
    col = 1
    for value, error in line:
        worksheet.write(row, col+0, value)
        worksheet.write(row, col+1, '+/-')
        worksheet.write(row, col+2, error)
        col += 3
    row += 1
    
### add bar graphs 

# create bar new chart of type 'bar'
strength_chart = workbook.add_chart({'type': 'column'})

# add chart title and axis labels
strength_chart.set_title({ 'name': 'Measured tensile strengths of selected materials'})
strength_chart.set_x_axis({'name': 'Material name'})
strength_chart.set_y_axis({'name': 'Tensile strength [MPa]'})

# add data to chart
strength_chart.add_series({
    'values':     '=Data!$B$2:$B$'+str(len(data)+1),
    'categories': '=Data!$A$2:$A$'+str(len(data)+1),
    'y_error_bars': {
        'type':         'custom',
        'plus_values':  '=Data!$D$2:$D$'+str(len(data)+1),
        'minus_values': '=Data!$D$2:$D$'+str(len(data)+1),
    },
    'data_labels': {
        'value': True,
        'position': 'inside_base',
        'font': { 'color': 'white' },
        'num_format': '#,###.'
    },
})

# unset legend
strength_chart.set_legend({'none': True})

# create chartsheet for storing the chart
chartsheet_1 = workbook.add_chartsheet('strengths plot')

# insert chart into chartsheet
chartsheet_1.set_chart(strength_chart)

# add bar graphs for Young's modulus and elongation at break
# ...

# close workbook
workbook.close()

Here is how our table and our barchart look like (after some minor manual formatting):

![](./img/lecture10/xlsx_table_02.png)  
![](./img/lecture10/xlsx_chart_02.png)

**Note**  
XlsxWriter can **only create new files**.  
It **cannot read or modify existing files**.  
Therefore, input data should be generated on the fly, e.g. by batch processing a directory tree.

**Further reading**  
Find out all about creating Excel charts at [https://xlsxwriter.readthedocs.io](https://xlsxwriter.readthedocs.io)